In [24]:
#Importando bibliotecas
import tensorflow as tf
from keras.datasets import mnist
from tensorflow.keras import layers , models
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
print("Bibliotecas Importadas")

Bibliotecas Importadas


In [25]:
# Carregando o Dataset
(x_train,y_train),(x_test,y_test)= mnist.load_data()

In [26]:
#Verificando o Formato da imagem 1 Layer, 60000 imagens formato de 28 x 28
x_train.shape

(60000, 28, 28)

In [27]:
#Normalizando os pixels de cada imagem + Ajustando os elementos (Reshape) para as nossas redes neurais
x_train = x_train.reshape((60000,28,28,1)).astype("float32") / 255 #
x_test = x_test.reshape((10000,28,28,1)).astype("float32") / 255

In [28]:
x_train.shape

(60000, 28, 28, 1)

In [29]:
y_test[0] #Agora é apenas uma label direta, uam forma boa de classificar seria usanto a to_categorical

7

In [30]:
# Como vamos usar Classentropy categorical, vamos deixar as labels de forma categórica
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [31]:
len(y_test[0]) # Vira um array de tamanho 10, onde o número classificado recebe o rótulo 1

10

In [32]:
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

### Definindo uma cnn

In [33]:
#Importando pois não quero usar o .layers pertencente ao sequencial
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [34]:
model1 = models.Sequential()
#Layer1
model1.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.2))
#Layer2
model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.2))
#Adicionando o Flatten
model1.add(Flatten())
#Adicionando o Dense para classificar entre os 10 digitos
model1.add(Dense(128, activation='relu'))
model1.add(Dense(10,activation='softmax'))

In [35]:
#Compilando a Cnn
model1.compile(optimizer="ADAM", loss="categorical_crossentropy", metrics=['accuracy'])

In [36]:
#Treinando a CNN
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 13, 13, 32)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 5, 5, 64)         

In [37]:
h1 = model1.fit(x_train,y_train , batch_size = 128, epochs = 5, verbose = 1, validation_data = (x_test, y_test))

Epoch 1/5
469/469 [==============================] - 41s 85ms/step - loss: 0.2380 - accuracy: 0.9278 - val_loss: 0.0573 - val_accuracy: 0.9819
Epoch 2/5
469/469 [==============================] - 37s 78ms/step - loss: 0.0678 - accuracy: 0.9792 - val_loss: 0.0456 - val_accuracy: 0.9860
Epoch 3/5
469/469 [==============================] - 36s 77ms/step - loss: 0.0499 - accuracy: 0.9843 - val_loss: 0.0315 - val_accuracy: 0.9883
Epoch 4/5
469/469 [==============================] - 36s 77ms/step - loss: 0.0394 - accuracy: 0.9879 - val_loss: 0.0328 - val_accuracy: 0.9893
Epoch 5/5
469/469 [==============================] - 36s 77ms/step - loss: 0.0335 - accuracy: 0.9896 - val_loss: 0.0330 - val_accuracy: 0.9882


In [38]:
cnn_predict = model1.predict(x_test)

313/313 [==============================] - 2s 6ms/step


In [39]:
cnn_accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(cnn_predict, axis=1))
print(f"CNN Accuracy: {cnn_accuracy}")

CNN Accuracy: 0.9882


In [40]:
#Salvando o Modelo
model1.save('saved_models/model1.hdf5')

### Definindo uma resnet

In [41]:
#Criando o bloco Residual - Modelo Funcional
def residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=True):
    #Por padrão fazemos primeiro ajustes no output
    y = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)

    y = layers.Conv2D(filters, kernel_size=kernel_size, padding='same')(y)
    y = layers.BatchNormalization()(y)

    if conv_shortcut:
        x = layers.Conv2D(filters, kernel_size=1, strides=stride, padding='same')(x)
        x = layers.BatchNormalization()(x)

    y = layers.Add()([x, y])
    y = layers.Activation('relu')(y)
    return y

In [42]:
resnet_input = layers.Input(shape=(28, 28, 1))
resnet_output = residual_block(resnet_input, filters=64)
resnet_output = layers.GlobalAveragePooling2D()(resnet_output)
resnet_output = layers.Dense(10, activation='softmax')(resnet_output)

In [43]:
model2 = models.Model(resnet_input, resnet_output)

In [44]:
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_7 (Conv2D)              (None, 28, 28, 64)   640         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 28, 28, 64)  256         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 activation_2 (Activation)      (None, 28, 28, 64)   0           ['batch_normalization_3[0][

In [45]:
#Compilando Resnet
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
# Treinando o modelo
h2 = model2.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 189s 100ms/step - loss: 1.2921 - accuracy: 0.6072 - val_loss: 7.1165 - val_accuracy: 0.1048
Epoch 2/5
1875/1875 [==============================] - 189s 101ms/step - loss: 0.7384 - accuracy: 0.7969 - val_loss: 10.2560 - val_accuracy: 0.2133
Epoch 3/5
1875/1875 [==============================] - 189s 101ms/step - loss: 0.5345 - accuracy: 0.8529 - val_loss: 9.2125 - val_accuracy: 0.2439
Epoch 4/5
1875/1875 [==============================] - 190s 102ms/step - loss: 0.4338 - accuracy: 0.8805 - val_loss: 5.8175 - val_accuracy: 0.3527
Epoch 5/5
1875/1875 [==============================] - 190s 101ms/step - loss: 0.3680 - accuracy: 0.8977 - val_loss: 11.6245 - val_accuracy: 0.2854


In [51]:
resnet_predict = model2.predict(x_test)

313/313 [==============================] - 7s 22ms/step


In [65]:
resnet_accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(resnet_predict, axis=-1))
print(f"Resnet Accuracy: {resnet_accuracy}")

Resnet Accuracy: 0.2854


In [66]:
#Salvando o Modelo
model1.save('saved_models/model2.hdf5')

### Definindo uma FCN

In [67]:
model3 = models.Sequential() # Onde será feita a FCN Simples

In [68]:
#FCN se diferencia pelo fato do mapa de saida ter as mesmas dimensões de um mapa de entrada
model3.add(Flatten(input_shape=(28, 28, 1)))
model3.add(Dense(128, activation='relu'))
model3.add(Dense(10, activation='softmax'))

In [69]:
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model3.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2588 - accuracy: 0.9261 - val_loss: 0.1365 - val_accuracy: 0.9599
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1126 - accuracy: 0.9668 - val_loss: 0.1022 - val_accuracy: 0.9679
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0779 - accuracy: 0.9766 - val_loss: 0.0812 - val_accuracy: 0.9740
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0577 - accuracy: 0.9825 - val_loss: 0.0977 - val_accuracy: 0.9715
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0460 - accuracy: 0.9857 - val_loss: 0.0702 - val_accuracy: 0.9777


In [70]:
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 128)               100480    
                                                                 
 dense_7 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [71]:
fcn_predict = model3.predict(x_test)

313/313 [==============================] - 0s 1ms/step


In [72]:
fcn_accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(fcn_predict, axis=1))
print(f"FCN Accuracy: {fcn_accuracy}")

FCN Accuracy: 0.9777


In [73]:
#Salvando o Modelo
model3.save('saved_models/model3.hdf5')

### Realizando Ensemble = AVG MODEL -> https://github.com/bnsreenu/python_for_microscopists/blob/master/213-ensemble_sign_language.py#L180

In [156]:
from keras.models import load_model
from sklearn.metrics import accuracy_score
import numpy as np

In [157]:
# Carregar os modelos treinados
model1 = load_model('saved_models/model1.hdf5')
model2 = load_model('saved_models/model2.hdf5')
model3 = load_model('saved_models/model3.hdf5')

In [158]:
models = [model1, model2, model3]

In [159]:
# Fazer previsões usando cada modelo
predictions = [model.predict(x_test) for model in models]
predictions = np.array(predictions)

313/313 [==============================] - 0s 1ms/step


In [160]:
# Somar as previsões
ensemble_prediction = np.argmax(np.sum(predictions, axis=0), axis=1)
ensemble_prediction

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [161]:
y_test_f = np.argmax(y_test, axis=1) # O que antes estava categórico voltou a ser multilabel
y_test_f

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [162]:
ensemble_accuracy = accuracy_score(y_test_f, ensemble_prediction)

print('Accuracy = ', ensemble_accuracy)

Accuracy =  0.9891
